In [1]:
!pip install sympy numpy
!python navier_stokes_random.py


python3: can't open file '/content/navier_stokes_random.py': [Errno 2] No such file or directory


In [3]:
import sympy as sp
import numpy as np
import csv
import random
import os
from datetime import datetime

# Define symbols
t, x, y, z = sp.symbols('t x y z')
Re = sp.symbols('Re', positive=True)
Re_val = 100  # Example Reynolds number

# Basis functions pool (sin, cos, exp, polynomials)
def basis_funcs(var):
    return [
        sp.sin(sp.pi * var),
        sp.cos(sp.pi * var),
        sp.exp(-var),
        var,
        var**2,
        1
    ]

# Generate a random smooth function as sum of products of basis funcs over variables
def random_func(vars, max_terms=4):
    expr = 0
    for _ in range(random.randint(1, max_terms)):
        coeff = random.uniform(-2, 2)
        term = coeff
        for var in vars:
            f = random.choice(basis_funcs(var))
            term *= f
        expr += term
    return expr

# Prepare grid for numerical evaluation
N = 10
xx = np.linspace(0, 1, N)
yy = np.linspace(0, 1, N)
zz = np.linspace(0, 1, N)
tt = 0  # Fixed time for evaluation

# Create output directory with timestamp
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
output_dir = f'navier_stokes_output_{timestamp}'
os.makedirs(output_dir, exist_ok=True)

# Number of functions to generate
num_functions = 10000

for func_idx in range(1, num_functions + 1):
    # Generate u, v, w, p expressions
    vars_space = [x, y, z]
    vars_all = [x, y, z, t]

    u_expr = random_func(vars_space)
    v_expr = random_func(vars_space)
    w_expr = random_func(vars_space)
    p_expr = random_func(vars_all, max_terms=3)

    # Compute partial derivatives for residuals
    du_dt = sp.diff(u_expr, t)
    du_dx = sp.diff(u_expr, x)
    du_dy = sp.diff(u_expr, y)
    du_dz = sp.diff(u_expr, z)
    d2u_dx2 = sp.diff(u_expr, x, 2)
    d2u_dy2 = sp.diff(u_expr, y, 2)
    d2u_dz2 = sp.diff(u_expr, z, 2)

    dv_dt = sp.diff(v_expr, t)
    dv_dx = sp.diff(v_expr, x)
    dv_dy = sp.diff(v_expr, y)
    dv_dz = sp.diff(v_expr, z)
    d2v_dx2 = sp.diff(v_expr, x, 2)
    d2v_dy2 = sp.diff(v_expr, y, 2)
    d2v_dz2 = sp.diff(v_expr, z, 2)

    dw_dt = sp.diff(w_expr, t)
    dw_dx = sp.diff(w_expr, x)
    dw_dy = sp.diff(w_expr, y)
    dw_dz = sp.diff(w_expr, z)
    d2w_dx2 = sp.diff(w_expr, x, 2)
    d2w_dy2 = sp.diff(w_expr, y, 2)
    d2w_dz2 = sp.diff(w_expr, z, 2)

    dp_dx = sp.diff(p_expr, x)
    dp_dy = sp.diff(p_expr, y)
    dp_dz = sp.diff(p_expr, z)

    # Define residuals
    res_u = du_dt + u_expr * du_dx + v_expr * du_dy + w_expr * du_dz + dp_dx - (1/Re) * (d2u_dx2 + d2u_dy2 + d2u_dz2)
    res_v = dv_dt + u_expr * dv_dx + v_expr * dv_dy + w_expr * dv_dz + dp_dy - (1/Re) * (d2v_dx2 + d2v_dy2 + d2v_dz2)
    res_w = dw_dt + u_expr * dw_dx + v_expr * dw_dy + w_expr * dw_dz + dp_dz - (1/Re) * (d2w_dx2 + d2w_dy2 + d2w_dz2)

    # Lambdify expressions for numerical evaluation
    try:
        res_u_num = sp.lambdify((x, y, z, t), res_u.subs(Re, Re_val), 'numpy')
        res_v_num = sp.lambdify((x, y, z, t), res_v.subs(Re, Re_val), 'numpy')
        res_w_num = sp.lambdify((x, y, z, t), res_w.subs(Re, Re_val), 'numpy')

        u_num = sp.lambdify((x, y, z), u_expr, 'numpy')
        v_num = sp.lambdify((x, y, z), v_expr, 'numpy')
        w_num = sp.lambdify((x, y, z), w_expr, 'numpy')
        p_num = sp.lambdify((x, y, z, t), p_expr, 'numpy')
    except Exception as e:
        print(f"Error in lambdify for function {func_idx}: {e}")
        continue

    # Collect data rows
    data_rows = []
    residuals = []

    for xi in xx:
        for yi in yy:
            for zi in zz:
                try:
                    u_val = u_num(xi, yi, zi)
                    v_val = v_num(xi, yi, zi)
                    w_val = w_num(xi, yi, zi)
                    p_val = p_num(xi, yi, zi, tt)

                    ru = res_u_num(xi, yi, zi, tt)
                    rv = res_v_num(xi, yi, zi, tt)
                    rw = res_w_num(xi, yi, zi, tt)

                    res_norm = np.sqrt(ru**2 + rv**2 + rw**2)
                    residuals.append(res_norm)

                    data_rows.append([xi, yi, zi, u_val, v_val, w_val, p_val, ru, rv, rw, res_norm])
                except Exception as e:
                    print(f"Error in numerical evaluation at point ({xi}, {yi}, {zi}) for function {func_idx}: {e}")
                    continue

    # Write velocity, pressure, residual data to CSV
    data_filename = os.path.join(output_dir, f'navier_stokes_data_{func_idx}.csv')
    with open(data_filename, 'w', newline='') as f:
        writer = csv.writer(f)
        writer.writerow(['x', 'y', 'z', 'u', 'v', 'w', 'p', 'res_u', 'res_v', 'res_w', 'res_norm'])
        writer.writerows(data_rows)

    # Write summary stats of residuals
    residuals = np.array(residuals)
    summary = [
        ['Residual Norm', 'Min', 'Max', 'Mean'],
        ['res_norm', np.min(residuals), np.max(residuals), np.mean(residuals)]
    ]

    stats_filename = os.path.join(output_dir, f'navier_stokes_residual_stats_{func_idx}.csv')
    with open(stats_filename, 'w', newline='') as f:
        writer = csv.writer(f)
        writer.writerows(summary)

    # Export the generated functions to CSV (string expressions)
    functions_filename = os.path.join(output_dir, f'navier_stokes_functions_{func_idx}.csv')
    with open(functions_filename, 'w', newline='') as f:
        writer = csv.writer(f)
        writer.writerow(['Function', 'Expression'])
        writer.writerow(['u(x,y,z)', str(u_expr)])
        writer.writerow(['v(x,y,z)', str(v_expr)])
        writer.writerow(['w(x,y,z)', str(w_expr)])
        writer.writerow(['p(x,y,z,t)', str(p_expr)])

    print(f"Function {func_idx} processed and data exported.")

print(f"All {num_functions} functions processed. Data saved in '{output_dir}' directory.")


Function 1 processed and data exported.
Function 2 processed and data exported.
Function 3 processed and data exported.
Function 4 processed and data exported.
Function 5 processed and data exported.
Function 6 processed and data exported.
Function 7 processed and data exported.
Function 8 processed and data exported.
Function 9 processed and data exported.
Function 10 processed and data exported.
Function 11 processed and data exported.
Function 12 processed and data exported.
Function 13 processed and data exported.
Function 14 processed and data exported.
Function 15 processed and data exported.
Function 16 processed and data exported.
Function 17 processed and data exported.
Function 18 processed and data exported.
Function 19 processed and data exported.
Function 20 processed and data exported.
Function 21 processed and data exported.
Function 22 processed and data exported.
Function 23 processed and data exported.
Function 24 processed and data exported.
Function 25 processed and

KeyboardInterrupt: 

In [4]:
import sympy as sp
import numpy as np
import csv
import random
import os
from datetime import datetime

# Define symbols
t, x, y, z = sp.symbols('t x y z')
Re = sp.symbols('Re', positive=True)
Re_val = 100  # Example Reynolds number

# Basis functions pool (sin, cos, exp, polynomials)
def basis_funcs(var):
    return [
        sp.sin(sp.pi * var),
        sp.cos(sp.pi * var),
        sp.exp(-var),
        var,
        var**2,
        1
    ]

# Generate a random smooth function as sum of products of basis funcs over variables
def random_func(vars, max_terms=4):
    expr = 0
    for _ in range(random.randint(1, max_terms)):
        coeff = random.uniform(-2, 2)
        term = coeff
        for var in vars:
            f = random.choice(basis_funcs(var))
            term *= f
        expr += term
    return expr

# Prepare grid for numerical evaluation
N = 10
xx = np.linspace(0, 1, N)
yy = np.linspace(0, 1, N)
zz = np.linspace(0, 1, N)
tt = 0  # Fixed time for evaluation

# Create output directory with timestamp
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
output_dir = f'navier_stokes_output_{timestamp}'
os.makedirs(output_dir, exist_ok=True)

# Number of functions to generate
num_functions = 10000

for func_idx in range(1, num_functions + 1):
    # Generate u, v, w, p expressions
    vars_space = [x, y, z]
    vars_all = [x, y, z, t]

    u_expr = random_func(vars_space)
    v_expr = random_func(vars_space)
    w_expr = random_func(vars_space)
    p_expr = random_func(vars_all, max_terms=3)

    # Compute partial derivatives for residuals
    du_dt = sp.diff(u_expr, t)
    du_dx = sp.diff(u_expr, x)
    du_dy = sp.diff(u_expr, y)
    du_dz = sp.diff(u_expr, z)
    d2u_dx2 = sp.diff(u_expr, x, 2)
    d2u_dy2 = sp.diff(u_expr, y, 2)
    d2u_dz2 = sp.diff(u_expr, z, 2)

    dv_dt = sp.diff(v_expr, t)
    dv_dx = sp.diff(v_expr, x)
    dv_dy = sp.diff(v_expr, y)
    dv_dz = sp.diff(v_expr, z)
    d2v_dx2 = sp.diff(v_expr, x, 2)
    d2v_dy2 = sp.diff(v_expr, y, 2)
    d2v_dz2 = sp.diff(v_expr, z, 2)

    dw_dt = sp.diff(w_expr, t)
    dw_dx = sp.diff(w_expr, x)
    dw_dy = sp.diff(w_expr, y)
    dw_dz = sp.diff(w_expr, z)
    d2w_dx2 = sp.diff(w_expr, x, 2)
    d2w_dy2 = sp.diff(w_expr, y, 2)
    d2w_dz2 = sp.diff(w_expr, z, 2)

    dp_dx = sp.diff(p_expr, x)
    dp_dy = sp.diff(p_expr, y)
    dp_dz = sp.diff(p_expr, z)

    # Define residuals
    res_u = du_dt + u_expr * du_dx + v_expr * du_dy + w_expr * du_dz + dp_dx - (1/Re) * (d2u_dx2 + d2u_dy2 + d2u_dz2)
    res_v = dv_dt + u_expr * dv_dx + v_expr * dv_dy + w_expr * dv_dz + dp_dy - (1/Re) * (d2v_dx2 + d2v_dy2 + d2v_dz2)
    res_w = dw_dt + u_expr * dw_dx + v_expr * dw_dy + w_expr * dw_dz + dp_dz - (1/Re) * (d2w_dx2 + d2w_dy2 + d2w_dz2)

    # Lambdify expressions for numerical evaluation
    try:
        res_u_num = sp.lambdify((x, y, z, t), res_u.subs(Re, Re_val), 'numpy')
        res_v_num = sp.lambdify((x, y, z, t), res_v.subs(Re, Re_val), 'numpy')
        res_w_num = sp.lambdify((x, y, z, t), res_w.subs(Re, Re_val), 'numpy')

        u_num = sp.lambdify((x, y, z), u_expr, 'numpy')
        v_num = sp.lambdify((x, y, z), v_expr, 'numpy')
        w_num = sp.lambdify((x, y, z), w_expr, 'numpy')
        p_num = sp.lambdify((x, y, z, t), p_expr, 'numpy')
    except Exception as e:
        print(f"Error in lambdify for function {func_idx}: {e}")
        continue

    # Collect data rows
    data_rows = []
    residuals = []

    for xi in xx:
        for yi in yy:
            for zi in zz:
                try:
                    u_val = u_num(xi, yi, zi)
                    v_val = v_num(xi, yi, zi)
                    w_val = w_num(xi, yi, zi)
                    p_val = p_num(xi, yi, zi, tt)

                    ru = res_u_num(xi, yi, zi, tt)
                    rv = res_v_num(xi, yi, zi, tt)
                    rw = res_w_num(xi, yi, zi, tt)

                    res_norm = np.sqrt(ru**2 + rv**2 + rw**2)
                    residuals.append(res_norm)

                    data_rows.append([xi, yi, zi, u_val, v_val, w_val, p_val, ru, rv, rw, res_norm])
                except Exception as e:
                    print(f"Error in numerical evaluation at point ({xi}, {yi}, {zi}) for function {func_idx}: {e}")
                    continue

    # Write velocity, pressure, residual data to CSV
    data_filename = os.path.join(output_dir, f'navier_stokes_data_{func_idx}.csv')
    with open(data_filename, 'w', newline='') as f:
        writer = csv.writer(f)
        writer.writerow(['x', 'y', 'z', 'u', 'v', 'w', 'p', 'res_u', 'res_v', 'res_w', 'res_norm'])
        writer.writerows(data_rows)

    # Write summary stats of residuals
    residuals = np.array(residuals)
    summary = [
        ['Residual Norm', 'Min', 'Max', 'Mean'],
        ['res_norm', np.min(residuals), np.max(residuals), np.mean(residuals)]
    ]

    stats_filename = os.path.join(output_dir, f'navier_stokes_residual_stats_{func_idx}.csv')
    with open(stats_filename, 'w', newline='') as f:
        writer = csv.writer(f)
        writer.writerows(summary)

    # Export the generated functions to CSV (string expressions)
    functions_filename = os.path.join(output_dir, f'navier_stokes_functions_{func_idx}.csv')
    with open(functions_filename, 'w', newline='') as f:
        writer = csv.writer(f)
        writer.writerow(['Function', 'Expression'])
        writer.writerow(['u(x,y,z)', str(u_expr)])
        writer.writerow(['v(x,y,z)', str(v_expr)])
        writer.writerow(['w(x,y,z)', str(w_expr)])
        writer.writerow(['p(x,y,z,t)', str(p_expr)])

    print(f"Function {func_idx} processed and data exported.")

print(f"All {num_functions} functions processed. Data saved in '{output_dir}' directory.")


Function 1 processed and data exported.
Function 2 processed and data exported.
Function 3 processed and data exported.
Function 4 processed and data exported.
Function 5 processed and data exported.
Function 6 processed and data exported.
Function 7 processed and data exported.
Function 8 processed and data exported.
Function 9 processed and data exported.
Function 10 processed and data exported.
Function 11 processed and data exported.
Function 12 processed and data exported.
Function 13 processed and data exported.
Function 14 processed and data exported.
Function 15 processed and data exported.
Function 16 processed and data exported.
Function 17 processed and data exported.
Function 18 processed and data exported.
Function 19 processed and data exported.
Function 20 processed and data exported.
Function 21 processed and data exported.
Function 22 processed and data exported.
Function 23 processed and data exported.
Function 24 processed and data exported.
Function 25 processed and

KeyboardInterrupt: 

In [5]:
import sympy as sp
import numpy as np
import csv
import random
import os
from datetime import datetime

# Initialize symbols
t, x, y, z = sp.symbols('t x y z')
Re = sp.symbols('Re', positive=True)
Re_val = 100  # Example Reynolds number

# Define a comprehensive pool of basis functions
def basis_funcs(var):
    return [
        sp.sin(sp.pi * var),
        sp.cos(sp.pi * var),
        sp.exp(-var),
        sp.exp(var),
        sp.log(var + 1),
        sp.sqrt(var + 1),
        sp.tan(sp.pi * var),
        sp.asin(sp.sin(sp.pi * var)),
        sp.acos(sp.cos(sp.pi * var)),
        sp.atan(sp.tan(sp.pi * var)),
        sp.sinh(var),
        sp.cosh(var),
        sp.tanh(var),
        var,
        var**2,
        var**3,
        1
    ]

# Generate a random smooth function as sum of products of basis functions over variables
def random_func(vars, max_terms=4):
    expr = 0
    for _ in range(random.randint(1, max_terms)):
        coeff = random.uniform(-2, 2)
        term = coeff
        for var in vars:
            f = random.choice(basis_funcs(var))
            term *= f
        expr += term
    return expr

# Prepare grid for numerical evaluation
N = 10
xx = np.linspace(0, 1, N)
yy = np.linspace(0, 1, N)
zz = np.linspace(0, 1, N)
tt = 0  # Fixed time for evaluation

# Create output directory with timestamp
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
output_dir = f'navier_stokes_output_{timestamp}'
os.makedirs(output_dir, exist_ok=True)

# Number of functions to generate
num_functions = 10000

for func_idx in range(1, num_functions + 1):
    # Generate u, v, w, p expressions
    vars_space = [x, y, z]
    vars_all = [x, y, z, t]

    u_expr = random_func(vars_space)
    v_expr = random_func(vars_space)
    w_expr = random_func(vars_space)
    p_expr = random_func(vars_all, max_terms=3)

    # Compute partial derivatives for residuals
    du_dt = sp.diff(u_expr, t)
    du_dx = sp.diff(u_expr, x)
    du_dy = sp.diff(u_expr, y)
    du_dz = sp.diff(u_expr, z)
    d2u_dx2 = sp.diff(u_expr, x, 2)
    d2u_dy2 = sp.diff(u_expr, y, 2)
    d2u_dz2 = sp.diff(u_expr, z, 2)

    dv_dt = sp.diff(v_expr, t)
    dv_dx = sp.diff(v_expr, x)
    dv_dy = sp.diff(v_expr, y)
    dv_dz = sp.diff(v_expr, z)
    d2v_dx2 = sp.diff(v_expr, x, 2)
    d2v_dy2 = sp.diff(v_expr, y, 2)
    d2v_dz2 = sp.diff(v_expr, z, 2)

    dw_dt = sp.diff(w_expr, t)
    dw_dx = sp.diff(w_expr, x)
    dw_dy = sp.diff(w_expr, y)
    dw_dz = sp.diff(w_expr, z)
    d2w_dx2 = sp.diff(w_expr, x, 2)
    d2w_dy2 = sp.diff(w_expr, y, 2)
    d2w_dz2 = sp.diff(w_expr, z, 2)

    dp_dx = sp.diff(p_expr, x)
    dp_dy = sp.diff(p_expr, y)
    dp_dz = sp.diff(p_expr, z)

    # Define residuals
    res_u = du_dt + u_expr * du_dx + v_expr * du_dy + w_expr * du_dz + dp_dx - (1/Re) * (d2u_dx2 + d2u_dy2 + d2u_dz2)
    res_v = dv_dt + u_expr * dv_dx + v_expr * dv_dy + w_expr * dv_dz + dp_dy - (1/Re) * (d2v_dx2 + d2v_dy2 + d2v_dz2)
    res_w = dw_dt + u_expr * dw_dx + v_expr * dw_dy + w_expr * dw_dz + dp_dz - (1/Re) * (d2w_dx2 + d2w_dy2 + d2w_dz2)

    # Lambdify expressions for numerical evaluation
    try:
        res_u_num = sp.lambdify((x, y, z, t), res_u.subs(Re, Re_val), 'numpy')
        res_v_num = sp.lambdify((x, y, z, t), res_v.subs(Re, Re_val), 'numpy')
        res_w_num = sp.lambdify((x, y, z, t), res_w.subs(Re, Re_val), 'numpy')

        u_num = sp.lambdify((x, y, z), u_expr, 'numpy')
        v_num = sp.lambdify((x, y, z), v_expr, 'numpy')
        w_num = sp.lambdify((x, y, z), w_expr, 'numpy')
        p_num = sp.lambdify((x, y, z, t), p_expr, 'numpy')
    except Exception as e:
        print(f"Error in lambdify for function {func_idx}: {e}")
        continue

    # Collect data rows
    data_rows = []
    residuals = []

    for xi in xx:
        for yi in yy:
            for zi in zz:
                try:
                    u_val = u_num(xi, yi, zi)
                    v_val = v_num(xi, yi, zi)
                    w_val = w_num(xi, yi, zi)
                    p_val = p_num(xi, yi, zi, tt)

                    ru = res_u_num(xi, yi, zi, tt)
                    rv = res_v_num(xi, yi, zi, tt)
                    rw = res_w_num(xi, yi, zi, tt)

                    res_norm = np.sqrt(ru**2 + rv**2 + rw**2)
                    residuals.append(res_norm)

                    data_rows.append([xi, yi, zi, u_val, v_val, w_val, p_val, ru, rv, rw, res_norm])
                except Exception as e:
                    print(f"Error in numerical evaluation at point ({xi}, {yi}, {zi}) for function {func_idx}: {e}")
                    continue

    # Write velocity, pressure, residual data to CSV
    data_filename = os.path.join(output_dir, f'navier_stokes_data_{func_idx}.csv')
    with open(data_filename, 'w', newline='') as f:
        writer = csv.writer(f)
        writer.writerow(['x', 'y', 'z', 'u', 'v', 'w', 'p', 'res_u', 'res_v', 'res_w', 'res_norm'])
        writer.writerows(data_rows)

    # Write summary stats of residuals
    residuals = np.array(residuals)
    summary = [
        ['Residual Norm', 'Min', 'Max', 'Mean'],
        ['res_norm', np.min(residuals), np.max(residuals), np.mean(residuals)]
    ]

    stats_filename = os.path.join(output_dir, f'navier_stokes_residual_stats_{func_idx}.csv')
    with open(stats_filename, 'w', newline='') as f:
        writer = csv.writer(f)
        writer.writerows(summary)

    # Export the generated functions to CSV (string expressions)
    functions_filename = os.path.join(output_dir, f'navier_stokes_functions_{func_idx}.csv')
    with open(functions_filename, 'w', newline='') as f:
        writer = csv.writer(f)
        writer.writerow(['Function', 'Expression'])
        writer.writerow(['u(x,y,z)', str(u_expr)])
        writer.writerow(['v(x,y,z)', str(v_expr)])
        writer.writerow(['w(x,y,z)', str(w_expr)])
        writer.writerow(['p(x,y,z,t)', str(p_expr)])

    print(f"Function {func_idx} processed and data exported.")

print(f"All {num_functions} functions processed. Data saved in '{output_dir}' directory.")


<lambdifygenerated-9082>:2: RuntimeWarning: invalid value encountered in scalar divide
  return -0.0094977219370479*pi**2*z**3*sin(pi*y) + 0.0569863316222874*z*sin(pi*y) - 1.13663116888587*sqrt(t + 1)*tan(pi*y)*cosh(z)*arctan(tan(pi*x)) + (-1.04054113974045*y**3*z**3*sin(pi*x) - 0.214445197523836*z*tan(pi*x)*arctan(tan(pi*y)))*(0.0945907220015121*pi*exp(-y)*cos(pi*x)*arccos(cos(pi*z)) - 0.411785785517942*exp(-x)*tan(pi*y)*arccos(cos(pi*z)) + 0.924422755572211*exp(-x)*arcsin(sin(pi*z))) - 0.00823571571035885*pi**2*(tan(pi*y)**2 + 1)*exp(-x)*tan(pi*y)*arccos(cos(pi*z)) - 0.917161010159234*(-0.94977219370479*pi*z**3*cos(pi*y) + 0.411785785517942*pi*(tan(pi*y)**2 + 1)*exp(-x)*arccos(cos(pi*z)) - 0.0945907220015121*exp(-y)*sin(pi*x)*arccos(cos(pi*z)))*sinh(x)*arcsin(sin(pi*z))*arctan(tan(pi*y)) + (-2.84931658111437*z**2*sin(pi*y) + 0.0945907220015121*pi*exp(-y)*sin(pi*x)*sin(pi*z)/sqrt(1 - cos(pi*z)**2) + 0.411785785517942*pi*exp(-x)*sin(pi*z)*tan(pi*y)/sqrt(1 - cos(pi*z)**2) - 0.9244227555

Function 1 processed and data exported.


<lambdifygenerated-9087>:2: RuntimeWarning: invalid value encountered in scalar divide
  return 0.0459147400566597*y**3*exp(-x)*sin(pi*t)*arctan(tan(pi*z)) - 0.000849282940525899*pi**2*z*(tan(pi*x)**2 + 1)*tan(pi*x)*sinh(y) - 0.00042464147026295*z*tan(pi*x)*sinh(y) + (-1.36489975092434*sinh(z)*arccos(cos(pi*x))*arctan(tan(pi*y)) - 0.848689573803171*arccos(cos(pi*z))*arcsin(sin(pi*y)))*(0.042464147026295*z*tan(pi*x)*cosh(y) + 1.91139065785961*(1 - tanh(y)**2)*exp(-x)*sin(pi*z) + 0.839900558409625*pi*cos(pi*z)*arccos(cos(pi*x)) - 0.856380553324392*sinh(y)*tanh(z)) - 0.0382278131571922*(tanh(y)**2 - 1)*exp(-x)*sin(pi*z)*tanh(y) + (0.609367100194387*z**3*tanh(x)*arcsin(sin(pi*y)) + 1.66867526209114*sqrt(y + 1)*sinh(z) + 0.393361730238579*exp(x)*tanh(z))*(-0.856380553324392*(1 - tanh(z)**2)*cosh(y) - 0.839900558409625*pi*sin(pi*z)*arccos(cos(pi*x))*arctan(tan(pi*y)) + 0.042464147026295*tan(pi*x)*sinh(y) + 1.91139065785961*pi*exp(-x)*cos(pi*z)*tanh(y)) + (1/100)*(1.71276110664878*(tanh(z)**2

Function 2 processed and data exported.


<lambdifygenerated-9095>:2: RuntimeWarning: divide by zero encountered in scalar divide
  return -1.98715518763631*pi*x**2*sin(pi*y)*sinh(t)*arctan(tan(pi*z)) + 0.00489951386368254*y*(tanh(z)**2 - 1)*tanh(z)*arcsin(sin(pi*x)) - 0.00244975693184127*pi**2*y*sin(pi*x)*tanh(z)/sqrt(1 - sin(pi*x)**2) + 0.00244975693184127*pi**2*y*sin(pi*x)*cos(pi*x)**2*tanh(z)/(1 - sin(pi*x)**2)**(3/2) - 1/100*z**3*(-0.403274058069338*log(x + 1)/(y + 1)**(3/2) - 1.64574875590861*pi**2*cos(pi*y)/sqrt(1 - cos(pi*y)**2) + 1.64574875590861*pi**2*sin(pi*y)**2*cos(pi*y)/(1 - cos(pi*y)**2)**(3/2)) + 0.0161309623227735*z**3*sqrt(y + 1)/(x + 1)**2 - 0.0967857739366412*z*sqrt(y + 1)*log(x + 1) + 0.0987449253545167*z*arccos(cos(pi*y)) + (0.73858131665777*exp(x)*cos(pi*z)*cosh(y) - 0.761215958671245*exp(y)*log(x + 1)*arctan(tan(pi*z)))*(-0.244975693184127*y*(1 - tanh(z)**2)*arcsin(sin(pi*x)) + 4.83928869683206*z**2*sqrt(y + 1)*log(x + 1) - 4.93724626772584*z**2*arccos(cos(pi*y))) + (-0.244975693184127*pi*y*cos(pi*x)*ta

...

<lambdifygenerated-11516>:2: RuntimeWarning: invalid value encountered in scalar divide
  return -0.469828911919633*pi*z**2*exp(-t)*sin(pi*x)*arccos(cos(pi*y))/sqrt(1 - cos(pi*x)**2) + (0.763754374646309*exp(x)*log(y + 1) - 0.626558805839965*pi*exp(-y)*sin(pi*z))*(0.763754374646309*exp(x)*log(y + 1) - 0.626558805839965*exp(-y)*sin(pi*z)*arctan(tan(pi*x))) + (1/100)*(0.626558805839965*exp(-y)*sin(pi*z)*arctan(tan(pi*x)) + 0.763754374646309*exp(x)/(y + 1)**2) + (0.626558805839965*exp(-y)*sin(pi*z)*arctan(tan(pi*x)) + 0.763754374646309*exp(x)/(y + 1))*(-1.89001035137563*y*tanh(x)*arcsin(sin(pi*z)) + 1.88173295331023*sqrt(y + 1)*sqrt(z + 1)*sinh(x) - 0.71032729674796*sin(pi*y)*tanh(x)) - 0.626558805839965*pi*(0.497825359860616*y*log(z + 1)*tanh(x) - 0.939154409129609*cos(pi*z)*tan(pi*x) - 0.576790751820713*exp(-z)*tan(pi*x))*exp(-y)*cos(pi*z)*arctan(tan(pi*x)) - 0.00763754374646309*exp(x)*log(y + 1) - 0.00626558805839965*pi**2*exp(-y)*sin(pi*z)*arctan(tan(pi*x))
<lambdifygenerated-11517>:2

Function 349 processed and data exported.


<lambdifygenerated-11523>:2: RuntimeWarning: invalid value encountered in scalar divide
  return 0.700839984150552*y**3*z**3*(-1.50018406813871*pi*(tan(pi*z)**2 + 1)*log(y + 1)*arcsin(sin(pi*x)) - 0.489468861300121*pi*sin(pi*z)*tanh(x)*arccos(cos(pi*y)) + 0.800387718082813*sinh(x)*sinh(y))*arccos(cos(pi*x)) - 0.0160077543616562*z*sinh(x)*sinh(y) + (1/100)*pi**2*(3.00036813627742*(tan(pi*z)**2 + 1)*log(y + 1)*tan(pi*z)*arcsin(sin(pi*x)) + 0.489468861300121*cos(pi*z)*tanh(x)*arccos(cos(pi*y))) - 0.00978937722600242*(tanh(x)**2 - 1)*cos(pi*z)*tanh(x)*arccos(cos(pi*y)) + (0.800387718082813*z*sinh(x)*sinh(y) - 1.50018406813871*log(y + 1)*tan(pi*z)*arcsin(sin(pi*x)) + 0.489468861300121*cos(pi*z)*tanh(x)*arccos(cos(pi*y)))*(0.800387718082813*z*sinh(y)*cosh(x) + 0.489468861300121*(1 - tanh(x)**2)*cos(pi*z)*arccos(cos(pi*y)) - 1.50018406813871*pi*log(y + 1)*cos(pi*x)*tan(pi*z)/sqrt(1 - sin(pi*x)**2)) + (0.800387718082813*z*sinh(x)*cosh(y) - 1.50018406813871*tan(pi*z)*arcsin(sin(pi*x))/(y + 1) +

Function 350 processed and data exported.


KeyboardInterrupt: 

In [7]:
import sympy as sp
import random

# Define symbols
x, y, z, t = sp.symbols('x y z t')
variables = [x, y, z, t]

# Function pool with required number of arguments
function_pool = {
    sp.sin: 1,
    sp.cos: 1,
    sp.exp: 1,
    sp.log: 1,
    sp.sqrt: 1,
    sp.besselj: 2,
    sp.bessely: 2,
    sp.besseli: 2,
    sp.besselk: 2,
    sp.hankel1: 2,
    sp.hankel2: 2,
    sp.jn: 2,
    sp.yn: 2
}


def generate_expression(variables, depth=3):
    expr = 0
    for _ in range(random.randint(1, depth)):
        func, arg_count = random.choice(list(function_pool.items()))
        coeff = random.uniform(-10, 10)

        try:
            if arg_count == 1:
                var = random.choice(variables)
                term = coeff * func(var)
            elif arg_count == 2:
                order = random.randint(0, 5)
                var = random.choice(variables)
                term = coeff * func(order, var)
            else:
                continue  # Skip functions with unsupported arg counts
            expr += term
        except Exception as e:
            print(f"Error generating term with {func}: {e}")
            continue
    return expr


def analyze_expression(expr):
    derivatives = {str(var): sp.diff(expr, var) for var in variables}
    integrals = {str(var): sp.integrate(expr, var) for var in variables}
    simplified = sp.simplify(expr)
    return derivatives, integrals, simplified
import csv
import os
from datetime import datetime

def export_to_csv(data, filename):
    with open(filename, 'w', newline='') as f:
        writer = csv.writer(f)
        for key, value in data.items():
            writer.writerow([key, value])

def main():
    num_expressions = 10  # Adjust as needed
    output_dir = f'sympy_expressions_{datetime.now().strftime("%Y%m%d_%H%M%S")}'
    os.makedirs(output_dir, exist_ok=True)

    for i in range(1, num_expressions + 1):
        expr = generate_expression(variables)
        derivatives, integrals, simplified = analyze_expression(expr)

        # Export original expression
        with open(os.path.join(output_dir, f'expression_{i}.txt'), 'w') as f:
            f.write(str(expr))

        # Export derivatives
        export_to_csv(derivatives, os.path.join(output_dir, f'derivatives_{i}.csv'))

        # Export integrals
        export_to_csv(integrals, os.path.join(output_dir, f'integrals_{i}.csv'))

        # Export simplified expression
        with open(os.path.join(output_dir, f'simplified_{i}.txt'), 'w') as f:
            f.write(str(simplified))

        print(f'Expression {i} processed and exported.')

if __name__ == "__main__":
    main()


Expression 1 processed and exported.
Expression 2 processed and exported.
Expression 3 processed and exported.
Expression 4 processed and exported.
Expression 5 processed and exported.
Expression 6 processed and exported.
Expression 7 processed and exported.
Expression 8 processed and exported.
Expression 9 processed and exported.
Expression 10 processed and exported.
